# LAB 2: Genetic Algorithms. 
### Pygmo Optimization for Himmelblau, Rastrigin and Rosenbrock Functions
### AI and Machine Learning // Suchkova Natalia М8О-114М-22
30.10.2022 @ MAI IT-Center

## Table of Contents

1. [**Self-adaptive Differential Evolution (sade)**](#gen1)



2. [**Grey Wolf Optimizer (gwo)**](#gen2)



3. [**Extended Ant Colony Optimization algorithm (gaco)**](#gen3)


4. [**Own Defined Optimization Problem**](#gen4)



In [1]:
import pygmo as pg
from pygmo import *
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
pg.__version__

'2.13.0'

<a id='gen1'></a>

## Self-adaptive Differential Evolution (SADE)

### Himmelblau

**Функция Химмельблау**	$f(x,y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2 $

In [3]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/3b6d888196974f862467cfe02748f3ae2ffdf476")

In [4]:
class HimmelblauOptimization: 
    """
    Задаем проблему Химмельблау, так как не нашла ее в pygmo почему-то :/
    This class defines a PyGMO-compatible User-Defined Optimization Problem.
    Конструктор  класса
    """

    def __init__(self,
                 x_min: float,
                 x_max: float,
                 y_min: float,
                 y_max: float): 
        
        # задаем границы определения функции

        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max

    def get_bounds(self):
        return ([self.x_min, self.y_min], [self.x_max, self.y_max])

    def fitness(self, x: list): # вычисляем значение функции
        
        function_value = (x[0] * x[0] + x[1] - 11.0) ** 2 + (x[0] + x[1] * x[1] - 7.0) ** 2
        return [function_value]

In [141]:
print(problem(HimmelblauOptimization(-5.12, 5.12, -5.12, 5.12)))

Problem name: <class '__main__.HimmelblauOptimization'>
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5.12, -5.12]
	Upper bounds: [5.12, 5.12]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none



In [5]:
problem_Him = problem(HimmelblauOptimization(-5.12, 5.12, -5.12, 5.12))

In [7]:
pop_size = 100  # зададим внешние переменные ака характеристики популяции,
current_seed = 23 # чтобы условия у всех адгоритмов были одинаковые
number_of_evolutions = 10

In [8]:
def few_evols(algo, pop, gens=10, 
              number_of_evols=10):
    """
    algo (pygmo obj) - алгоритм для оптимизации
    pop (pygmo obj) - популяция
    number_of_evols (int) - количество раз эволюции
    """
    cur_algo = algorithm(algo(gen=gens)) # задаем параметры алгоритма
    cur_algo.set_verbosity(2)  # частота лагирования
    individuals_list = []
    fitness_list = []
    # Заставляем популяцию эволюционировать
    for i in range(number_of_evols):
        pop = cur_algo.evolve(pop)  # запускаем эволюцию  начальной популяции
        individuals_list.append(pop.get_x()[pop.best_idx()])
        fitness_list.append(pop.get_f()[pop.best_idx()])
    
    log = cur_algo.extract(algo).get_log() # можно высети лог (историю лагирования) каждые n итераций из set_verbosity()
    return pop, individuals_list, fitness_list, log



'''

1/ добавить норм вывод
2. добавить визуализацию

3. доавить датафрейм с результатами
# Вытащим лучших индивидов из всех поколений
best_x = [ind[0] for ind in individuals_list]
best_y = [ind[1] for ind in individuals_list]

# Выразим границы задачи
(x_min, y_min), (x_max, y_max) = udp.get_bounds()

# Изобразим минимизацию за все поколения
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(np.arange(0, number_of_evolutions), fitness_list, label='Значение функции')

# Выведем лучший образец
champion_n = np.argmin(np.array(fitness_list))
ax.scatter(champion_n, np.min(fitness_list), marker='x', color='r', label='Чемпион за всё время')

# Оформление
ax.set_xlim((0, number_of_evolutions))
ax.grid('major')
ax.set_title('Лучший представитель для каждого поколения', fontweight='bold')
ax.set_xlabel('Номер поколения')
ax.set_ylabel(r'Значение функции Химмельблау $f(x,y)$')
ax.legend(loc='upper right')
ax.set_yscale('log')
plt.tight_layout()

# Показ
plt.show()


# Изобразим функцию Химмельблау
grid_points = 100
x_vector = np.linspace(x_min, x_max, grid_points)
y_vector = np.linspace(y_min, y_max, grid_points)
x_grid, y_grid = np.meshgrid(x_vector, y_vector)
z_grid = np.zeros((grid_points, grid_points))
for i in range(x_grid.shape[1]):
    for j in range(x_grid.shape[0]):
        z_grid[i, j] = udp.fitness([x_grid[i, j], y_grid[i, j]])[0]

# Create figure
fig, ax = plt.subplots(figsize=(9,5))
cs = ax.contour(x_grid, y_grid, z_grid, 50)

# Показываем лучших представителей за каждое поколение
ax.scatter(best_x, best_y, marker='x', color='r')

# Оформление
ax.set_xlim((x_min, x_max))
ax.set_ylim((y_min, y_max))
ax.set_title('Функция Химмельблау', fontweight='bold')
ax.set_xlabel('X')
ax.set_ylabel('Y')
cbar = fig.colorbar(cs)
cbar.ax.set_ylabel(r'Значение функции Химмельблау $f(x,y)$')
plt.tight_layout()

# Показ
plt.show()
'''

In [150]:
# можно еще посмотреть информацию обалгоритмах
algo1 = algorithm(sade(gen=10))
print(algo1)

Algorithm name: saDE: Self-adaptive Differential Evolution [stochastic]
	Thread safety: basic

Extra info:
	Generations: 10
	Variant: 2
	Self adaptation variant: 1
	Stopping xtol: 1e-06
	Stopping ftol: 1e-06
	Memory: false
	Verbosity: 0
	Seed: 2053364609


In [16]:
pop_him = population(problem_Him, size=pop_size, seed=current_seed)

pop_him = few_evols(sade, pop_him)
pop_him[1]  # individuals_list 
pop_him[2]  # fitness_list

[array([0.01574637]),
 array([0.01479609]),
 array([0.01479609]),
 array([0.00696351]),
 array([0.00156061]),
 array([0.00042323]),
 array([0.00042323]),
 array([1.77233755e-05]),
 array([1.08741282e-05]),
 array([3.84186758e-08])]

In [17]:
pop_him[0].champion_x

array([3.00001464, 1.99994817])

In [18]:
pop_him[0].champion_f

array([3.84186758e-08])

In [19]:
pop_him[3] # выводим лог

[(1,
  100,
  1.0874128164929134e-05,
  0.22188336266088843,
  0.4802374716656514,
  4.493621180730514,
  0.039150086887246485),
 (3,
  300,
  1.2479699082784465e-06,
  0.7296741165126628,
  0.34761425424494585,
  0.021369314632310177,
  0.006311542497788462),
 (5,
  500,
  1.2479699082784465e-06,
  0.7296741165126628,
  0.34761425424494585,
  0.021369314632310177,
  0.006311542497788462),
 (7,
  700,
  1.2479699082784465e-06,
  0.7296741165126628,
  0.34761425424494585,
  0.021369314632310177,
  0.006311542497788462),
 (9,
  900,
  9.637053337038284e-07,
  0.16467530915871262,
  0.9252918256657917,
  12.071178376275899,
  0.0013476604922223142)]

### Rastrigin

**Функция Растригина** $f(x) = An + \displaystyle\sum_{i=1}^{n} \big\lbrack  x_i^2 - Acos(2pix_i) \big\rbrack$, where $A = 10$

In [20]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/0ef36b9a7230888e739f302e1a5d67e36c152de8")

In [46]:
# classics
sade_algo = algorithm(sade(gen=10))
problem_Rast = problem(rastrigin(dim=2)) 
pop = population(problem_Rast, size=pop_size, seed=current_seed)
pop = sade_algo.evolve(pop)
pop.champion_x

array([0.01091344, 0.00698779])

In [47]:
problem_Rast = problem(rastrigin(dim=2))  # задаем параметры оптимизируемой функции "проблемы" Растиригина

In [142]:
print(problem_Rast)

Problem name: Rastrigin Function
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5.12, -5.12]
	Upper bounds: [5.12, 5.12]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 2
	Has hessians: true
	User implemented hessians sparsity: true
	Expected hessian components: [2]

	Fitness evaluations: 0
	Gradient evaluations: 0
	Hessians evaluations: 0

	Thread safety: basic



In [55]:
pop_rast = population(problem_Rast, size=pop_size, seed=current_seed)  # задаем параметры начальной популяции

pop_rast = few_evols(sade, pop_rast, 10, 3)

In [56]:
pop_rast[0].champion_x  # лучший результат - индивид

array([ 0.00284147, -0.00429281])

In [57]:
pop_rast[0].champion_f  # лучший результат - значение функции

array([0.00525756])

In [58]:
pop_rast[3]

[(1,
  100,
  0.6353954428006752,
  0.4426384244115158,
  0.9896920417573147,
  2.3361709090625173,
  14.53085389177109),
 (3,
  300,
  0.09709159338132167,
  0.8603567536901959,
  0.20701206331410818,
  2.287379395799285,
  15.069157741190443),
 (5,
  500,
  0.09709159338132167,
  0.8603567536901959,
  0.20701206331410818,
  2.287379395799285,
  15.069157741190443),
 (7,
  700,
  0.0052575554680522885,
  0.42970094146564,
  0.8297445196486171,
  2.2548602522774632,
  15.160991779103712),
 (9,
  900,
  0.0052575554680522885,
  0.42970094146564,
  0.8297445196486171,
  2.2548602522774632,
  15.160991779103712)]

### Rosenbrock

**Функция Розенброка**	$f(x) = \displaystyle\sum_{i=1}^{n-1} \big\lbrack 100(x_{i+1} - x_i^2)^2 +(x_i - 1)^2 \big\rbrack$

In [59]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/53f008f82897059e3591d0a9a0aa27b60a1745b4")

In [60]:
problem_Rosen = problem(rosenbrock(dim=2)) # задаем проблему Розенброка

In [61]:
pop_rosen = population(problem_Rosen, size=pop_size, seed=current_seed)
pop_rosen = few_evols(sade, pop_rosen)

In [62]:
pop_rosen[0]

Problem name: Multidimensional Rosenbrock Function
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5, -5]
	Upper bounds: [10, 10]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 2
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 10100
	Gradient evaluations: 0

	Thread safety: constant

Population size: 100

List of individuals: 
#0:
	ID:			1251045422705684049
	Decision vector:	[1.00003, 1.00008]
	Fitness vector:		[2.71005e-08]
#1:
	ID:			13306745122587129611
	Decision vector:	[1.00158, 1.00307]
	Fitness vector:		[3.45565e-06]
#2:
	ID:			2243283290487074856
	Decision vector:	[1.00102, 1.00199]
	Fitness vector:		[1.26185e-06]
#3:
	ID:			14933863474924020957
	Decision vector:	[1.00027, 1.0008]
	Fitness vector:		[7.25206e-06]
#4:
	ID:			17243

In [63]:
pop_rosen[0].champion_x

array([1.00007685, 1.00014839])

In [64]:
pop_rosen[0].champion_f

array([8.72375668e-09])

<a id='gen2'></a>

## Grey Wolf Optimizer (gwo)

### Himmelblau

In [65]:
pop_him = population(problem_Him, size=pop_size, seed=current_seed)  # задаем параметры начальной популяции

pop_him = few_evols(gwo, pop_him)

In [66]:
pop_him[0].champion_x 

array([-2.80264294,  3.12996135])

In [67]:
pop_him[0].champion_f

array([0.00026781])

### Rastrigin

In [116]:
pop_rast = population(problem_Rast, size=pop_size, seed=current_seed)

pop_rast = few_evols(gwo, pop_rast, 5, 2)

In [117]:
pop_rast

(Problem name: Rastrigin Function
 	Global dimension:			2
 	Integer dimension:			0
 	Fitness dimension:			1
 	Number of objectives:			1
 	Equality constraints dimension:		0
 	Inequality constraints dimension:	0
 	Lower bounds: [-5.12, -5.12]
 	Upper bounds: [5.12, 5.12]
 	Has batch fitness evaluation: false
 
 	Has gradient: true
 	User implemented gradient sparsity: false
 	Expected gradients: 2
 	Has hessians: true
 	User implemented hessians sparsity: true
 	Expected hessian components: [2]
 
 	Fitness evaluations: 1100
 	Gradient evaluations: 0
 	Hessians evaluations: 0
 
 	Thread safety: basic
 
 Population size: 100
 
 List of individuals: 
 #0:
 	ID:			1251045422705684049
 	Decision vector:	[4.55588e-08, -7.08233e-08]
 	Fitness vector:		[1.40687e-12]
 #1:
 	ID:			13306745122587129611
 	Decision vector:	[7.72134e-08, -7.15263e-08]
 	Fitness vector:		[2.19558e-12]
 #2:
 	ID:			2243283290487074856
 	Decision vector:	[5.98136e-08, -9.39822e-08]
 	Fitness vector:		[2.46203e-12]
 #3:


In [118]:
pop_rast[0].champion_x

array([ 4.55588446e-08, -7.08233009e-08])

In [119]:
pop_rast[0].champion_f

array([1.40687462e-12])

### Rosenbrock

In [120]:
pop_rosen = population(problem_Rosen, size=pop_size, seed=current_seed)

pop_rosen = few_evols(gwo, pop_rosen)

In [121]:
pop_rosen[0].champion_f

array([1.20365663e-05])

In [122]:
pop_rosen[0].champion_x

array([1.00345807, 1.00690011])

In [123]:
pop_rosen[3]

[(1, 9.736981953246515e-05, 9.667586289746562e-05, 9.644561310497547e-05),
 (3, 9.736981953246515e-05, 9.667586289746562e-05, 9.644561310497547e-05),
 (5, 9.736981953246515e-05, 9.667586289746562e-05, 9.644561310497547e-05),
 (7, 9.736981953246515e-05, 9.667586289746562e-05, 9.644561310497547e-05),
 (9, 1.3262246923776149e-05, 4.52959606194109e-05, 9.644561310497547e-05)]

<a id='gen3'></a>

## Extended Ant Colony Optimization algorithm (gaco)
сделаю третий, так как интересно как работает, почему бы и нет (:

без вспомогательной функции просто посмотрим на результаты

### Himmelblau

In [126]:
gaco_algo = algorithm(gaco(10, 13, 1.0, 1e9, 0.0, 1, 7, 100000, 100000, 0.0, False, 23))
gaco_algo.set_verbosity(3)

In [127]:
number_of_evolutions = 2
pop = population(problem_Him, size=pop_size, seed=current_seed)
for i in range(number_of_evolutions):
    pop = gaco_algo.evolve(pop)

In [128]:
pop.champion_x

array([2.99999988, 2.00000004])

In [129]:
pop.champion_f

array([4.87947353e-13])

### Rastrigin

In [130]:
pop = population(problem_Rast, size=pop_size, seed=current_seed)
pop = gaco_algo.evolve(pop)

In [131]:
pop.champion_x # хорошо справился и за 1 круг эволюции

array([0.0048657 , 0.00990002])

In [132]:
pop.champion_f

array([0.02413484])

### Rosenbrock

In [136]:
pop = population(problem_Rosen, size=pop_size, seed=current_seed)
for i in range(number_of_evolutions): # 1 цикла эволюции не хватило, 2 достаточно
    pop = gaco_algo.evolve(pop)

In [137]:
pop.champion_x

array([1.00047668, 1.00123554])

In [138]:
pop.champion_f

array([8.1768731e-06])

<a id='gen4m'></a>

## Special Task - Own Defined Optimization Problem